In [1]:
import numpy as np
import math
from matplotlib.pyplot import cm
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
# plt.rcParams.update({'font.size': 20,'figure.figsize': (10.0, 6.0)})
import ipywidgets as wg
from ipywidgets import Layout
# %matplotlib widget
%matplotlib qt
import time
import csv
import matplotlib
from scipy.signal import savgol_filter


## Van Duzer formulas for the magnetic and kinetic Inductance 
![](Picture1.jpg "formulas")


In [177]:
#Classes and Functions

class Inductor: 
    
    #Constants
    mu0= 1.25663706e-6 #permeability free space
    h=6.62607004e-34   #Plank's Constant
    kb=1.3806485e-23   #Boltzman Constant
    hbar=h/(2*np.pi)    
    
    #Attributes
    def __init__(self, name, t, w, l, N, ro, Tc, Jc):
        self.name=name #Name of inductor
        self.t=t*1e-9 #Thickness in nanometer
        self.w=w*1e-9 #width in nanometer
        self.l=l*1e-6 #length in micrometer
        self.ro=ro*1e-8 #normal state resistivity in ohm m
        self.Tc=Tc #Critical Temperature in K
        self.N=N #number of turns
        self.Jc=Jc #Critical Current Density
        self.Ic=self.Jc*self.w*self.t
        self.AspRatio=self.w/self.t

    def __str__(self):
        desc="\n" +self.name+":\n Thickness= "+str(round(self.t*1e9,2))+" nm \n Width= "+str(round(self.w*1e9,2))+" nm \n "+\
            "Length= "+str(round(self.l*1e6,2))+" um \n Aspect Ratio= "+str(round(self.AspRatio,2))+"\n Resistivity= "+ str(round(self.ro*1e8,2))+" u.ohm cm\n Critical"+\
            " Temperature= "+ str(self.Tc)+" K"+"\n Area= "+str(round(self.Area()*1e12,5))+" um2"+"\n Sheet Inductance= "+\
            str(round(self.Ls()*1e12,2)) +" pH\n Ratio Lk/Lm= "+ str(round(self.ratioLkLm(),2))+"\n"+\
            " Ic= "+str(round(self.Ic*1e3,3))+" mm\n"
        return desc
    
    
    def G(self):
        G= 3*self.w ##Gap between fingers
        return G
    
    
    def lam(self):
        lam= self.G()+self.w  #lambda/ period
        return lam 
    
    
    def delta(self): 
        delta= 1.764*kb*self.Tc  #Order parameter/ Energy 
        return delta
    
    
    def Lm(self): #van duzer See Figure 1 first equation [Magnetic Inductance]
        Lm=self.l*(mu0/(2*self.w))*(((self.lambdaa()/2)*math.sinh(2*self.t/self.lambdaa())-self.t)/((math.sinh(self.t/self.lambdaa()))**2))
        return Lm


    def Lk(self): #van duzer See Figure 1 second equation [Kintetic Inductance]
        Lk=self.l*(mu0/(2*self.w))*(((self.lambdaa()/2)*math.sinh(2*self.t/self.lambdaa())+self.t)/((math.sinh(self.t/self.lambdaa()))**2))
        return Lk

    
    def lambdaa(self): ## Pentration depth from Critical Temp and Normal State Resistivity
        lambda_m=np.sqrt((hbar*self.ro)/(np.pi*delta*mu0))
        return lambda_m

    
    def Area(self): #Total area of the inductor
        L=(self.l/self.N)-self.G()
        area= L*self.N*self.lam()
        return area

    def Ls(self): #Sheet Inductance
        Ls= (self.Lk()+self.Lm())*self.w/self.l
        return Ls
    
    def Lt(self): #Total Inductance of a single inductor
        return self.Lk()+self.Lm()
    
    def ratioLkLm(self): #Ratio of Lk/Lm for one inductor
        return self.Lk()/self.Lm()        


def ratioL2L1(L2,L1): #Ratio of L2/L1 
    ratio=L2/L1
    return ratio

def plotcurr(I,I_1,I_2): #Plotting Currents vs Applied Current
    plot1= plt.plot(I,I_1,linestyle='--',c=c)
    plot2= plt.plot(I,I_2,linestyle='dashdot',label='$r= $ %d' % (ratio),c=c)
    plt.legend(loc ="lower right")
    plt.xlim(0,3)
    plt.ylim(0,2)
    plt.xlabel('$I_{tot}/I_{c1}$')
    plt.ylabel('$I_i/I_{c1}$')
    return plot1,plot2
    

def plotind(I,L_1,L_2,L): #Plotting Inductances vs Applied Current
    plot1=plt.plot(I,L_1*1e12,linestyle='-',c=c)
    plot2=plt.plot(I,L_2*1e12,linestyle='-',c=c)
    plt.plot(I,L*1e12,linestyle='--',label='$r= $ %d' % (ratio),c=c )
    plt.xlim(0,4)
    plt.legend(loc ="upper right");        
    plt.xlabel('$I_{tot}/I_{c1}$');
    plt.ylabel('$L(J)$ [pH]');
    return plot1,plot2
    

## Interpolation for Clem's plot for Lk vs I 

![](Picture2.png "formulas")


In [16]:
##Interpolation to the above graph of the slow esperiments model 

w = []
x = []
y = []
    
with open('ClemData.csv', newline='') as f:
    reader = csv.reader(f, delimiter=' ')
    
    for row in reader:
            w.extend(row)
    
           
for i in range(len(w)):
    if i%2==1:
        y.append(float(w[i]))
    else:
        x.append(float(w[i]))

        
#Test 

z=[0.3,0.5,0.9]
tun= 1/np.interp(z, x, y)
print(tun)


[1.03778407 1.1215706  2.03705961]



$L_1=L_{m1}+L_{k1}$

$L_2=L_{m2}+L_{k2}$

$L_{tot}^{-1}=L_{1}^{-1}+L_{2}^{-1}$

$I_2=I_1 \frac{L_1}{L_2}$

$I_{tot}=I_1+I_2$ 

In [178]:
#Ratios Comparison 

L1=Inductor("L1", 20,100/30*0.25,1.1,1,247,12,1.5e11)
print(L1)
 
n=int(5)   #Number of different ratios you want to compare 
color = iter(cm.rainbow(np.linspace(0, 1, n)))

for i in range(n):
    
    L2=Inductor("L2",20,(200/(30*(i+1))),18,3*i+2,247,12,1.5e11) #Inductor 2
    ratio= ratioL2L1(L2.Lt(),L1.Lt())
    
    #Considering the effect of both magnetic and kinetic inductance [k+m] gives      
    I_1=np.arange(0,0.999,0.001) #I1/Ic1
    L_1=np.power(np.interp(I_1, x, y),-1)*L1.Lk()+L1.Lm() #Solving for L_1 when we add the magnetic inductance
    L_2=L2.Lk()*np.ones(np.shape(L_1))+L2.Lm() #Solving for L_2 when we add the magnetic inductance
    I_2=I_1*L_1/L_2 #Current in the second inductor
    I=I_1+I_2 #Total Current
    L=np.power(np.power(L_1,-1)+np.power(L_2,-1),-1) #Total Inductance (Parallel)
    
    fig0=plt.figure(0)
    c = next(color)        
    plotcurr(I,I_1,I_2) #Plotting Applied Current VS. Current at each inductor
    
    ratio=ratioL2L1(L2.Lt(),L1.Lt())   
    fig1=plt.figure(1)
    plotind(I,L_1,L_2,L) #Plotting Inductance VS. Current
    
    print("Case (ratio="+str(round(ratio,2))+")\n",L2)

plt.show()
print ("\nIc1="+str(round(L1.Ic*1e6,2))+"um")


L1:
 Thickness= 20.0 nm 
 Width= 0.83 nm 
 Length= 1.1 um 
 Aspect Ratio= 0.04
 Resistivity= 247.0 u.ohm cm
 Critical Temperature= 12 K
 Area= 0.00366 um2
 Sheet Inductance= 13.74 pH
 Ratio Lk/Lm= 1638.52
 Ic= 0.003 mm

Case (ratio=2.05)
 
L2:
 Thickness= 20.0 nm 
 Width= 6.67 nm 
 Length= 18.0 um 
 Aspect Ratio= 0.33
 Resistivity= 247.0 u.ohm cm
 Critical Temperature= 12 K
 Area= 0.47893 um2
 Sheet Inductance= 13.74 pH
 Ratio Lk/Lm= 1638.52
 Ic= 0.02 mm

Case (ratio=4.09)
 
L2:
 Thickness= 20.0 nm 
 Width= 3.33 nm 
 Length= 18.0 um 
 Aspect Ratio= 0.17
 Resistivity= 247.0 u.ohm cm
 Critical Temperature= 12 K
 Area= 0.23933 um2
 Sheet Inductance= 13.74 pH
 Ratio Lk/Lm= 1638.52
 Ic= 0.01 mm

Case (ratio=6.14)
 
L2:
 Thickness= 20.0 nm 
 Width= 2.22 nm 
 Length= 18.0 um 
 Aspect Ratio= 0.11
 Resistivity= 247.0 u.ohm cm
 Critical Temperature= 12 K
 Area= 0.15953 um2
 Sheet Inductance= 13.74 pH
 Ratio Lk/Lm= 1638.52
 Ic= 0.007 mm

Case (ratio=8.18)
 
L2:
 Thickness= 20.0 nm 
 Width= 1.67 

In [170]:
#adding the design without shunting path assuming it will reach 1.4x inductance 
I1=np.arange(0,0.99,0.001)
L11=np.power(np.interp(I1, x, y),-1)*L1.Lk()+L1.Lm()

plt.figure(0)
plt.plot(I1,L11*1e9,linestyle='--',label='$L_1$ 1.4x')
# plt.legend(loc ="upper left")


#worst case scenario 1.4x due to quantum fluctuations 
I1_pr=np.arange(0,0.75,0.001)
L1_pr=L11=np.power(np.interp(I1_pr, x, y),-1)*L1.Lk()+L1.Lm()
plt.plot(I1_pr,L1_pr*1e9)
plt.grid()


In [181]:
#Adding both together 

#Considering the effect of both magnetic and kinetic inductance [k+m] gives      
I_1=np.arange(0,0.99,0.001) #I1/Ic1
L_1=np.power(np.interp(I_1, x, y),-1)*L1.Lk()+L1.Lm()
L_2=L2.Lk()*np.ones(np.shape(L_1))+L2.Lm()
I_2=I_1*L_1/L_2
I=I_1+I_2
L=np.power(np.power(L_1,-1)+np.power(L_2,-1),-1)

plt.figure(0)
plotind(I,L_1,L_2,L)

#worst case scenario 1.4x due to quantum fluctuations 
I1_pr=np.arange(0,0.75,0.001)
L1_pr=np.power(np.interp(I1_pr, x, y),-1)*L1.Lk()+L1.Lm()
L2_pr=L2.Lk()*np.ones(np.shape(L1_pr))+L2.Lm()
I2_pr=I1_pr*L1_pr/L2_pr
I_pr=I1_pr+I2_pr
L_pr=np.power(np.power(L1_pr,-1)+np.power(L2_pr,-1),-1)

plt.figure(1)
plt.plot(I_pr,L1_pr*1e9,label='$L_1$ ')
plt.plot(I_pr,L2_pr*1e9,label='$L_2$ ')
plt.plot(I_pr,L_pr*1e9,label='$L_{tot}$')

#without shunting path assuming it will reach 1.4x inductance 
I_1=np.arange(0,0.99,0.001)
L11=np.power(np.interp(I_1, x, y),-1)*L1.Lk()+L1.Lm()

plt.figure(2)
plt.plot(I_1,L11*1e9,linestyle='-',label='$L_1$ 1.4x')
plt.legend(loc ="upper left")


#worst case scenario 1.4x due to quantum fluctuations 
I1_pr=np.arange(0,0.75,0.001)
L1_pr=L11=np.power(np.interp(I1_pr, x, y),-1)*L1.Lk()+L1.Lm()
plt.plot(I1_pr,L1_pr*1e9)



In [182]:
##Shunting path freq and inductance calculations 

#best case scenario 
I_1=np.arange(0,0.99,0.001) #I1/Ic1
L_1=np.power(np.interp(I_1, x, y),-1)*L1.Lk()+L1.Lm()
L_2=L2.Lk()*np.ones(np.shape(L_1))+L2.Lm()
I_2=I_1*L_1/L_2
I=I_1+I_2
L=np.power(np.power(L_1,-1)+np.power(L_2,-1),-1)
print('SHUNTING path:\n\nMaximum measured DIVERGENT inductance=', max(L)*1e9, 'nH,\nTunability [total inductance]=' , max(L)/min(L), 'x, Tunability of L1=',max(L_1)/min(L_1),'x' )


#worst case scenario 
I1_pr=np.arange(0,0.75,0.001)
L1_pr=np.power(np.interp(I1_pr, x, y),-1)*L1.Lk()+L1.Lm()
L2_pr=L2.Lk()*np.ones(np.shape(L1_pr))+L2.Lm()
I2_pr=I1_pr*L1_pr/L2_pr
I_pr=I1_pr+I2_pr
L_pr=np.power(np.power(L1_pr,-1)+np.power(L2_pr,-1),-1)
print('\nMaximum measured inductance=', max(L_pr)*1e9, 'nH,\nTunability [total inductance]=' , max(L_pr)/min(L_pr), 'x, Tunability of L1= 1.4 x' )



## No Shunting path
#best case scenario
I_1=np.arange(0,0.99,0.001)
L11=np.power(np.interp(I_1, x, y),-1)*L1.Lk()+L1.Lm()
print('\nNO SHUNTING path:\n\nMaximum measured DIVERGENT inductance=', max(L11)*1e9, 'nH,\nTunability=' , max(L11)/min(L11), 'x')


#worst case scenario
I1_pr=np.arange(0,0.75,0.001)
L1_pr=L11=np.power(np.interp(I1_pr, x, y),-1)*L1.Lk()+L1.Lm()
print('\nMaximum measured inductance=', max(L1_pr)*1e9, 'nH,\nTunability=' , max(L1_pr)/min(L1_pr), 'x')


SHUNTING path:

Maximum measured DIVERGENT inductance= 0.12201925886339658 nH,
Tunability [total inductance]= 3.2731658032354636 x, Tunability of L1= 5.540012694534231 x

Maximum measured inductance= 0.049382512778212914 nH,
Tunability [total inductance]= 1.324685575122542 x, Tunability of L1= 1.4 x

NO SHUNTING path:

Maximum measured DIVERGENT inductance= 0.24369933043555325 nH,
Tunability= 5.540012694534231 x

Maximum measured inductance= 0.06188856852170902 nH,
Tunability= 1.4069117655926038 x


## Actual Design--> 30nm:

In [184]:
L1=Inductor("L1", 30,670,400,12,247,12,1.5e11)
L2=Inductor("L2", 30,3300,3800,17,247,12,1.5e11)

# L=np.power(np.power(L1.Lt(),-1)+np.power(L2.Lt(),-1),-1)

I_1=np.arange(0,0.999,0.001) #I1/Ic1
L_1=np.power(np.interp(I_1, x, y),-1)*L1.Lk()+L1.Lm() #Solving for L_1 when we add the magnetic inductance
L_2=L2.Lk()*np.ones(np.shape(L_1))+L2.Lm() #Solving for L_2 when we add the magnetic inductance
I_2=I_1*L_1/L_2 #Current in the second inductor
I=I_1+I_2 #Total Current
L=np.power(np.power(L_1,-1)+np.power(L_2,-1),-1) #Total Inductance (Parallel)
        
print(L1,L2)

plt.figure(0)
plotcurr(I,I_1,I_2)
plt.figure(1)
plotind(I,L_1,L_2,L)


L1:
 Thickness= 30.0 nm 
 Width= 670.0 nm 
 Length= 400.0 um 
 Aspect Ratio= 22.33
 Resistivity= 247.0 u.ohm cm
 Critical Temperature= 12 K
 Area= 1007.3584 um2
 Sheet Inductance= 9.16 pH
 Ratio Lk/Lm= 728.45
 Ic= 3.015 mm
 
L2:
 Thickness= 30.0 nm 
 Width= 3300.0 nm 
 Length= 3800.0 um 
 Aspect Ratio= 110.0
 Resistivity= 247.0 u.ohm cm
 Critical Temperature= 12 K
 Area= 47938.44 um2
 Sheet Inductance= 9.16 pH
 Ratio Lk/Lm= 728.45
 Ic= 14.85 mm



([<matplotlib.lines.Line2D at 0x7fdf52d4a250>],
 [<matplotlib.lines.Line2D at 0x7fdf52d4a5b0>])